In [ ]:
import os
from dotenv import load_dotenv
from datetime import datetime
import matplotlib.pyplot as plt
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from finquant.portfolio import build_portfolio
from alpaca_trade_api import REST
from alpaca_trade_api.rest import TimeFrame

# Load API keys from environment variables
load_dotenv("key.env")
alpaca_api_key = os.getenv("Alpaca_Paper_Key_ID")
alpaca_secret_key = os.getenv("Alpaca_Paper_Secret_Key")

# Create Alpaca REST client
alpaca = REST(alpaca_api_key, alpaca_secret_key, base_url='https://paper-api.alpaca.markets')

names = []

new_name = ''

# Start a loop that will run until the user enters 'exit'.
while new_name != 'exit':
    # Ask the user for a stock ticker
    new_name = input("Please add stock ticker, or enter 'exit': ")

    # Add the new name to our list
    if new_name != 'exit':
        names.append(new_name)

year = int(input('Enter a year for start date: '))   # year for start date
month = int(input('Enter a month for start date: ')) # month for start date
day = int(input('Enter a day for start date: '))     # day for start date

# Use datetime for start_date and end_date
start_date = datetime(year, month, day)
end_date = datetime.now()

# Use build_portfolio from finquant with yahoo finance as data api
pf = build_portfolio(names=names, start_date=start_date, end_date=end_date, data_api="yfinance")

pf.comp_cumulative_returns().plot()
plt.axhline(y=0, color="black", lw=3)
plt.title('Cumulative Returns')
plt.show()

# Perform and plot results of Monte Carlo run (5000 iterations)
opt_w, opt_res = pf.mc_optimisation(num_trials=5000)
print(opt_w)
print(opt_res)

# Plot the results of the Monte Carlo optimization
pf.mc_plot_results()

# Plot the Efficient Frontier
ef = EfficientFrontier(pf.comp_mean_returns(freq=252), risk_models.sample_cov(pf.comp_returns(freq=252)))
pf.ef.plot_efficient_frontier()
plt.show()

# Plot optimal portfolios based on Efficient Frontier
pf.ef.plot_optimal_portfolios()
plt.show()

print(' ')
print('Optimized portfolio results')
# Create an instance of EfficientFrontier
ef = EfficientFrontier(pf.comp_mean_returns(freq=1), pf.comp_cov())

# Perform optimization for minimum volatility
min_volatility = ef.min_volatility()
print(min_volatility)

# Print relevant quantities of the optimized portfolio
expected_return, volatility, sharpe_ratio = ef.portfolio_performance(verbose=True)

# Get target allocations for maximum sharpe ratio and minimum volatility
target_allocation_maximum_sharpe = ef.max_sharpe_ratio()
target_allocation_minimum_volatility = ef.min_volatility()

print(target_allocation_maximum_sharpe)
print(target_allocation_minimum_volatility)

# Get Alpaca account information
account = alpaca.get_account()
print(account)

# Rebalance the portfolio on Alpaca based on target allocations
def alpaca_rebalance(target_allocations):
    positions = alpaca.list_positions()

    for position in positions:
        symbol = position.symbol
        if symbol not in target_allocations:
            alpaca.close_position(symbol)
        else:
            target_allocation = target_allocations[symbol]
            current_allocation = position.market_value / float(account.equity)
            diff = target_allocation - current_allocation

            if diff > 0:
                target_value = float(account.equity) * target_allocation
                target_quantity = target_value / float(pf.price[symbol][-1])
                alpaca.submit_order(
                    symbol=symbol,
                    qty=target_quantity - float(position.qty),
                    side='buy',
                    type='market',
                    time_in_force='gtc'
                )
            elif diff < 0:
                target_value = float(account.equity) * target_allocation
                target_quantity = target_value / float(pf.price[symbol][-1])
                alpaca.submit_order(
                    symbol=symbol,
                    qty=float(position.qty) - target_quantity,
                    side='sell',
                    type='market',
                    time_in_force='gtc'
                )

# Define target allocations for the portfolio
target_allocations = {
    'AAPL': 0.3,
    'GOOGL': 0.2,
    'MSFT': 0.4,
    'TSLA': 0.1
}

# Rebalance the portfolio on Alpaca
alpaca_rebalance(target_allocations)


In [6]:
#!pip install PyPortfolioOpt

In [7]:
import os
from dotenv import load_dotenv
from datetime import datetime
import matplotlib.pyplot as plt
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from finquant.portfolio import build_portfolio
from alpaca_trade_api import REST
from alpaca_trade_api.rest import TimeFrame

# Load API keys from environment variables
load_dotenv("key.env")
alpaca_api_key = os.getenv("Alpaca_Paper_Key_ID")
alpaca_secret_key = os.getenv("Alpaca_Paper_Secret_Key")

# Create Alpaca REST client
alpaca = REST(alpaca_api_key, alpaca_secret_key, base_url='https://paper-api.alpaca.markets')

names = []

new_name = ''

# Start a loop that will run until the user enters 'exit'.
while new_name != 'exit':
    # Ask the user for a stock ticker
    new_name = input("Please add stock ticker, or enter 'exit': ")

    # Add the new name to our list
    if new_name != 'exit':
        names.append(new_name)

year = int(input('Enter a year for start date: '))   # year for start date
month = int(input('Enter a month for start date: ')) # month for start date
day = int(input('Enter a day for start date: '))     # day for start date

# Use datetime for start_date and end_date
start_date = datetime(year, month, day)
end_date = datetime.now()

# Use build_portfolio from finquant with yahoo finance as data api
pf = build_portfolio(names=names, start_date=start_date, end_date=end_date, data_api="yfinance")

pf.comp_cumulative_returns().plot()
plt.axhline(y=0, color="black", lw=3)
plt.title('Cumulative Returns')
plt.show()

# Perform and plot results of Monte Carlo run (5000 iterations)
opt_w, opt_res = pf.mc_optimisation(num_trials=5000)
print(opt_w)
print(opt_res)

# Plot the results of the Monte Carlo optimization
pf.mc_plot_results()

# Plot the Efficient Frontier
ef = EfficientFrontier(pf.comp_mean_returns(freq=252), risk_models.sample_cov(pf.comp_returns(freq=252)))
pf.ef.plot_efficient_frontier()
plt.show()

# Plot optimal portfolios based on Efficient Frontier
pf.ef.plot_optimal_portfolios()
plt.show()

print(' ')
print('Optimized portfolio results')
# Create an instance of EfficientFrontier
ef = EfficientFrontier(pf.comp_mean_returns(freq=1), pf.comp_cov())

# Perform optimization for minimum volatility
min_volatility = ef.min_volatility()
print(min_volatility)

# Print relevant quantities of the optimized portfolio
expected_return, volatility, sharpe_ratio = ef.portfolio_performance(verbose=True)

# Get target allocations for maximum sharpe ratio and minimum volatility
target_allocation_maximum_sharpe = ef.max_sharpe_ratio()
target_allocation_minimum_volatility = ef.min_volatility()

print(target_allocation_maximum_sharpe)
print(target_allocation_minimum_volatility)

# Get Alpaca account information
account = alpaca.get_account()
print(account)

# Rebalance the portfolio on Alpaca based on target allocations
def alpaca_rebalance(target_allocations):
    positions = alpaca.list_positions()

    for position in positions:
        symbol = position.symbol
        if symbol not in target_allocations:
            alpaca.close_position(symbol)
        else:
            target_allocation = target_allocations[symbol]
            current_allocation = position.market_value / float(account.equity)
            diff = target_allocation - current_allocation

            if diff > 0:
                target_value = float(account.equity) * target_allocation
                target_quantity = target_value / float(pf.price[symbol][-1])
                alpaca.submit_order(
                    symbol=symbol,
                    qty=target_quantity - float(position.qty),
                    side='buy',
                    type='market',
                    time_in_force='gtc'
                )
            elif diff < 0:
                target_value = float(account.equity) * target_allocation
                target_quantity = target_value / float(pf.price[symbol][-1])
                alpaca.submit_order(
                    symbol=symbol,
                    qty=float(position.qty) - target_quantity,
                    side='sell',
                    type='market',
                    time_in_force='gtc'
                )

# Define target allocations for the portfolio
target_allocations = {
    'AAPL': 0.3,
    'GOOGL': 0.2,
    'MSFT': 0.4,
    'TSLA': 0.1
}

# Rebalance the portfolio on Alpaca
alpaca_rebalance(target_allocations)


ModuleNotFoundError: No module named 'dotenv'